In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.7 MB/s eta 0:00:00


In [ ]:
!pip install pdf2image pytesseract pillow
!pip install tesseract-ocr tesseract-ocr-bos # za Linux


  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tesseract-ocr-bos (from versions: none)
ERROR: No matching distribution found for tesseract-ocr-bos


In [ ]:
import pdfplumber

# Putanja do PDF fajla
pdf_path = "/content/SS 01.pdf"

# Naziv tekstualne izlazne datoteke
output_path = "izvod_tekst.txt"

start_keyword = "U PRVOM LICU"
end_keyword = "PORTRET SLIKARA"

collected_text = ""
collecting = False

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        lines = text.split("\n")
        for line in lines:
            if start_keyword in line:
                collecting = True
            if collecting:
                collected_text += line + " "
            if end_keyword in line:
                collecting = False
                break

# Ukloni višestruke razmake i očisti tekst
import re
cleaned_text = re.sub(r"\s+", " ", collected_text).strip()

# Spasi u txt datoteku
with open(output_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"Izvučeni tekst je sačuvan u: {output_path}")


Izvučeni tekst je sačuvan u: izvod_tekst.txt


# P

In [ ]:
import re

# Učitaj ulazni tekst (pretpostavljam da si već pročitao iz fajla)
with open("/content/SS 15-16.txt", "r", encoding="utf-8") as f:
    tekst = f.read()

# Podjela teksta po autorima (ime i prezime, svaki autor je odvojen praznim redom)
autori_blokovi = re.split(r"\n(?=[A-ZČĆŽŠĐ][a-zčćžšđ]+\s+[A-ZČĆŽŠĐ][a-zčćžšđ]+\n)", tekst)

clanci = []
broj = 1

for blok in autori_blokovi:
    linije = blok.strip().split("\n")
    if len(linije) < 2:
        continue

    autor = linije[0].strip()
    sadrzaj = linije[1:]

    trenutni_naslov = ""
    trenutni_tekst = []

    for linija in sadrzaj:
        # Novačlanak: linija sve velikim slovima (naslov)
        if linija.isupper():
            if trenutni_naslov and trenutni_tekst:
                tekst_clanka = "\n".join(trenutni_tekst).strip()
                clanak = (
                    "<***>\n"
                    "NOVINA: Sarajevske sveske 15-16\n"
                    "DATUM: 2007\n"
                    "RUBRIKA: N/A\n"
                    "NADNASLOV: N/A\n"
                    f"NASLOV: {trenutni_naslov}\n"
                    "PODNASLOV: N/A\n"
                    "STRANA: N/A\n"
                    f"AUTOR(I): {autor}\n\n"
                    f"{tekst_clanka}\n"
                )
                clanci.append(clanak)
                broj += 1
            trenutni_naslov = linija.strip()
            trenutni_tekst = []
        else:
            # Ukloni brojeve koji bi mogli predstavljati oznake (ali ne iz riječi ili rečenica)
            linija = re.sub(r'^\d+\s*', '', linija)
            trenutni_tekst.append(linija)

    # Dodaj posljednji članak
    if trenutni_naslov and trenutni_tekst:
        tekst_clanka = "\n".join(trenutni_tekst).strip()
        clanak = (
            "<***>\n"
            "NOVINA: Sarajevske sveske 15-16\n"
            "DATUM: 2007\n"
            "RUBRIKA: N/A\n"
            "NADNASLOV: N/A\n"
            f"NASLOV: {trenutni_naslov}\n"
            "PODNASLOV: N/A\n"
            "STRANA: N/A\n"
            f"AUTOR(I): {autor}\n\n"
            f"{tekst_clanka}\n"
        )
        clanci.append(clanak)

# Zapiši rezultat u izlazni fajl
with open("clanci_output.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(clanci))


In [ ]:
import re

def ocisti_tekst(tekst):
    # Ukloni linije koje izgledaju kao "|", "| 83", "|     ", itd.
    tekst = re.sub(r"^\|\s*\d*\s*$", "", tekst, flags=re.MULTILINE)
    tekst = tekst.replace("|", "")
    # Ukloni sve pojave naslova "SARAJEVSKE SVESKE Nº 47/48"
    tekst = re.sub(r"SARAJEVSKE SVESKE Nº 21/22", "", tekst, flags=re.IGNORECASE)

    # Ukloni brojeve stranica koji se nalaze između praznih redova
    tekst = re.sub(r"\n\d{1,4}\n", "\n", tekst)

    # Ukloni višestruke prazne redove
    tekst = re.sub(r"\n{2,}", "\n\n", tekst)

    return tekst.strip()



In [ ]:
with open("clanci_output.txt", "r", encoding="utf-8") as f:
    sadrzaj = f.read()

# Čišćenje osnovnog sadržaja
sadrzaj = sadrzaj.replace("\f", "\n")
sadrzaj = ocisti_tekst(sadrzaj)

with open("clanci_ocisceni.txt", "w", encoding="utf-8") as f:
    f.write(sadrzaj)


In [ ]:
# Naziv ulazne i izlazne datoteke
ulazna_datoteka = "/content/clanci_ocisceni.txt"
izlazna_datoteka = "ocisceno.txt"

# Otvori ulaznu datoteku i pročitaj sadržaj
with open(ulazna_datoteka, "r", encoding="utf-8") as f:
    sadrzaj = f.read()

# Ukloni sve znakove '|'
sadrzaj_ociscen = sadrzaj.replace("|", "")

# Snimi očišćeni sadržaj u novu datoteku
with open(izlazna_datoteka, "w", encoding="utf-8") as f:
    f.write(sadrzaj_ociscen)

print("Znakovi '|' su uspješno uklonjeni.")


Znakovi '|' su uspješno uklonjeni.


In [ ]:
# Otvaramo fajl i čitamo sadržaj
with open("ocisceno.txt", "r", encoding="utf-8") as file:
    sadrzaj = file.read()

# 1. Ukloni sve pojave izraza
sadrzaj = sadrzaj.replace("SARAJEVSKE SVESKE Nº 21/22", "")
with open("ocisceno.txt", "w", encoding="utf-8") as file:
        file.write(sadrzaj)



In [ ]:
import re

# Regex za datume tipa '24. april', '. april', '5. maj' itd.
mjeseci = r"januar|februar|mart|april|maj|juni|jul|august|septembar|oktobar|novembar|decembar"
datum_regex = re.compile(rf"(\b\d{{1,2}}\.)?\s*\.?\s*({mjeseci})\b", flags=re.IGNORECASE)

# Regex za izbacivanje svega unutar <...>
zagrade_regex = re.compile(r"<[^>]*>")

with open("ocisceno.txt", "r", encoding="utf-8") as f:
    sadrzaj = f.read()

# Podjela po člancima
clanci = sadrzaj.strip().split("<***>")

ocisceni_clanci = []

for clanak in clanci:
    clanak = clanak.strip()
    if not clanak:
        continue

    dijelovi = clanak.split("\n\n", 1)
    if len(dijelovi) != 2:
        continue

    metapodaci = dijelovi[0].strip()
    tekst = dijelovi[1].replace("\n", " ").strip()

    # Ukloni izraze poput '24. april', '. april', '5. novembar'
    tekst = datum_regex.sub("", tekst)

    # Ukloni izraze u obliku <...>
    tekst = zagrade_regex.sub("", tekst)

    # Ukloni višestruke razmake
    tekst = re.sub(r"\s{2,}", " ", tekst).strip()

    finalni = "<***>\n" + metapodaci + "\n\n" + tekst + "\n"
    ocisceni_clanci.append(finalni)

# Snimi rezultat
with open("rezultat.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(ocisceni_clanci))

print("✅ Gotovo! Datum i izrazi u zagradama uklonjeni, format sačuvan.")


✅ Gotovo! Datum i izrazi u zagradama uklonjeni, format sačuvan.


In [82]:
import re

# Učitaj sadržaj iz .txt fajla
with open("/content/Sarajevske sveske-sve.txt", "r", encoding="utf-8") as f:
    sadrzaj = f.read()

# Podijeli članke po separatoru <***>
clanci = sadrzaj.strip().split("<***>")

ocisceni_clanci = []

for clanak in clanci:
    clanak = clanak.strip()
    if not clanak:
        continue

    # Podijeli članak na metapodatke i tekst članka (razdvojeno praznim redom)
    dijelovi = clanak.split("\n\n", 1)
    if len(dijelovi) != 2:
        continue

    metapodaci = dijelovi[0].strip()
    tekst_clanka = dijelovi[1].strip()

    # Pretvori više redova teksta članka u jedan red
    tekst_clanka_u_jednom_redu = " ".join(tekst_clanka.splitlines())
    tekst_clanka_u_jednom_redu = re.sub(r"\s{2,}", " ", tekst_clanka_u_jednom_redu).strip()

    # Sastavi članak ponovo u traženom formatu
    finalni = "<***>\n" + metapodaci + "\n\n" + tekst_clanka_u_jednom_redu + "\n"
    ocisceni_clanci.append(finalni)

# Spoji sve članke i upiši u novi fajl
with open("ocisceno.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(ocisceni_clanci))

print("✅ Završen zadatak: svi tekstovi članaka su sada u jednoj liniji, bez narušavanja formata.")


✅ Završen zadatak: svi tekstovi članaka su sada u jednoj liniji, bez narušavanja formata.
